In [ ]:
! pip install diffusers huggingface_hub transformers accelerate --upgrade

In [ ]:
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from google.colab import drive
from tqdm import tqdm

from os import path
import random
import torch
import numpy as np

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Set random seeds to 42 for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

In [ ]:
PROMPT_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes.txt'
OPPOSITE_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes_opposites.txt'
SAVE_PATH  = f'/content/drive/My Drive/CLIP_Facial_Impressions/sd2_images'

In [ ]:
prompt_attributes = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(PROMPT_PATH).read().split('\n') if line}
opposite_prompts = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(OPPOSITE_PATH).read().split('\n') if line}


In [ ]:
model_id = "stabilityai/stable-diffusion-2"

scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

In [ ]:
seeds = [random.randint(0,100000) for _ in range(100)]
seedstr = '\n'.join([str(i) for i in seeds])
with open(f'/content/drive/My Drive/CLIP_Facial_Impressions/sd2_seeds.txt', 'w') as f:
  f.write(seedstr)

In [ ]:
for attribute in prompt_attributes.keys():
  print(attribute)

  positive_prompt = prompt_attributes[attribute]
  negative_prompt = opposite_prompts[attribute]

  for i in tqdm(range(25)):
    image = pipe(positive_prompt).images[0]
    image.save(path.join(SAVE_PATH,f'{attribute}_positive_{i}.png'))

  for i in tqdm(range(25)):
    image = pipe(negative_prompt).images[0]
    image.save(path.join(SAVE_PATH,f'{attribute}_negative_{i}.png'))